# Tratamiento de datos COVID-10
## Objeto
Se tratarán los datos de la [Johns Hopkins](https://github.com/CSSEGISandData/COVID-19) que se traem al directorio $data_dir. 

Se utilizarán tablas de población para, en el futuro, obtener información más relevante

## Obtención de datos
Se obtienen los datos de población desde los repositorios de Naciones Unidas.
Los datos de contagios y fallecidos por COVID-19 se traen del repositorio mantenido por la Universidad Johns Hopkins, 
los de vacunaciones, de Our World In Data, los de medidas, del Centro Europeo para la Prevención y el Control de Enfermedades 
(una agencia de la UE),

In [430]:
data_dir = './data'
jhopkins_dir = 'COVID-19'
timeseries_dir = 'csse_covid_19_data/csse_covid_19_time_series'
covid_git_url = 'https://github.com/CSSEGISandData/COVID-19'
population_file = 'WPP2019_PopulationByAgeSex_Medium.csv'
covid_confirmed_file = 'time_series_covid19_confirmed_global.csv'
covid_deaths_file = 'time_series_covid19_deaths_global.csv'
un_pop_repo_url = 'https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/' + population_file
covid_owid_dir = 'covid-19-data'
covid_owid_url = 'https://github.com/owid/covid-19-data.git'
covid_owid_vaccinations_file = 'public/data/vaccinations/vaccinations.csv'
measures_file = 'measures.csv'

! if [ ! -d '{data_dir}' ]; then echo 'No existe directorio de datos. Creándolo...'; mkdir '{data_dir}' ; fi
! if [ ! -d '{data_dir}/{jhopkins_dir}/{timeseries_dir}' ]; then echo No existe clone del repositorio '{jhopkins_dir}/{timeseries_dir}' Ejecutando git clone '{covid_git_url}'; cd '{data_dir}';  git clone '{covid_git_url}'; fi
! if [ ! -d "{data_dir}/{covid_owid_dir}" ]; then echo 'No existe clone del repositorio {covid_owid_dir}. Ejecutando git clone {covid_owid_url}'; cd '{data_dir}';  git clone '{covid_owid_url}'; fi
! if [ ! -f '{data_dir}/{population_file}' ]; then echo 'No existen datos de población Trayéndolos de {un_pop_repo_url}'; cd '{data_dir}'; curl -g '{un_pop_repo_url}' -o $population_file; fi

### Datos de población

In [431]:
! ls -l '{data_dir}/{population_file}'

-rw-r--r-- 1 10001 10001 118543941 Feb  7 05:26 ./data/WPP2019_PopulationByAgeSex_Medium.csv


### Datos de la Johns Hopkins

In [432]:
! cd '{data_dir}/{jhopkins_dir}' ; git pull
# ! cd '{data_dir}/{jhopkins_dir}' ; git checkout origin/master -- '{timeseries_dir}'
! ls -l '{data_dir}/{jhopkins_dir}/{timeseries_dir}/' 

Already up to date.
total 17172
-rw-r--r-- 1 10001 10001 2531245 Aug  7 06:23 Errata.csv
-rw-r--r-- 1 10001 10001    2155 Jul 28 17:35 README.md
-rw-r--r-- 1 10001 10001  763022 Aug  8 08:47 time_series_covid19_confirmed_global.csv
-rw-r--r-- 1 10001 10001 7735195 Aug  8 06:44 time_series_covid19_confirmed_US.csv
-rw-r--r-- 1 10001 10001  529683 Aug  8 08:47 time_series_covid19_deaths_global.csv
-rw-r--r-- 1 10001 10001 5321011 Aug  8 06:44 time_series_covid19_deaths_US.csv
-rw-r--r-- 1 10001 10001  687278 Aug  8 06:44 time_series_covid19_recovered_global.csv


### Datos de Our World In Data 

In [ ]:
! cd '{data_dir}/{covid_owid_dir}'; git pull
! ls -l '{data_dir}/{covid_owid_dir}/{covid_owid_vaccinations_file}'

Already up to date.


### Datos de medidas tomadas

In [ ]:
! curl 'https://www.ecdc.europa.eu/sites/default/files/documents/response_graphs_data_2021-01-28.csv' >> $data_dir/$measures_file
! ls -l $data_dir/$measures_file

### Importación de datos
Lo primero es común a todos los programas que traten datos

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np

Se utiliza Pandas para leer el fichero csv. Las columnas que notienen datos (las 4 primeras), se ponen de índice. 

Se van a leer los datos de fallecimientos, casos confirmados, población y vacunaciones.

Los datos de medidas, hasta que no se optimice el procedimiento, se obtienen después

In [ ]:
deaths = pd.read_csv(data_dir + '/' + jhopkins_dir + '/' +  timeseries_dir + '/' + covid_deaths_file, 
                      index_col=[0,1,2,3])
confirmed = pd.read_csv(data_dir + '/' + jhopkins_dir + '/' +  timeseries_dir + '/' + covid_confirmed_file, 
                           index_col=[0,1,2,3])
population = pd.read_csv (data_dir + '/' + population_file)
vaccinations = pd.read_csv (data_dir + '/' + covid_owid_dir + '/' + covid_owid_vaccinations_file, 
                            usecols=['location', 'date',
                                     'people_vaccinated_per_hundred',
                                     'people_fully_vaccinated_per_hundred'], 
                            parse_dates=['date'])
deaths.head(3)

## Tratamiento de datos
Se filtran los datos de población para suprimir la división por edades y quedarnos con los del año 2021. 
Se conserva la división por género por si tuviese interés

In [ ]:
pop_filtered = population[(population['VarID'] == 2) & (population['Time'] == 2021) ] \
                         [['Location','PopMale','PopFemale','PopTotal']] \
                         .groupby(['Location']).sum()

#population.head()
#pop_filtered.index.names

pop_filtered.loc['South Africa']
#pop_filtered.head(3)

Se cambia el nombre de algunos índices. Se ha visto que el nombre de los países es distinto en las tablas de la 
ONU que en las tablas COVID. Por ejemplo, los **Estados Unidos de América** aparecen en las tablas de la ONU
como **United States of America**, mientras que en las tablas COVID como **US**. El diccionario **change_dict**
está pensado para todos estos casos.

In [ ]:
change_pop_dict = {
    'United States of America': 'US',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Russian Federation': 'Russia',
    'China, Taiwan Province of China': 'Taiwan*'
              }

pop_index_list = pop_filtered.index.tolist()

for country in change_pop_dict:
    pop_index_list[pop_index_list.index(country)] = change_pop_dict[country]

pop_filtered.index = pop_index_list

In [ ]:
pop_filtered.loc[['US',
                  'Spain',
                  'Portugal',
                  'Morocco',
                  'United Kingdom',
                  'Ireland',
                  'Italy',
                  'Germany',
                  'France',
                  'Argentina',
                  'Chile',
                  'Mexico',
                  'Peru',
                  'Colombia', 
                  'Venezuela', 
                  'Cuba', 
                  'Bolivia', 
                  'Ecuador', 
                  'Guatemala', 
                  'Panama', 
                  'Costa Rica',
                  'Russia',
                  'China',
                  'Taiwan*',
                  'Belgium',
                  'Netherlands',
                  'Australia',
                  'Israel',
                  'India',
                  'South Africa',
                  'Japan',
                 ]].sort_values('PopTotal', ascending=False)

Se obtienen los dataframes de incrementos por población

In [ ]:
deaths_sum = deaths.groupby(level=1).sum()
confirmed_sum = confirmed.groupby(level=1).sum()


def get_by_pop_inc_df (df,pop):
    df_by_pop_inc = df.divide(pop_filtered['PopTotal']/(pop/1000), axis=0).transpose().diff()
    df_by_pop_inc.index = pd.to_datetime(df_by_pop_inc.index)
    return df_by_pop_inc
    
confirmed_factor = 100000
deaths_factor = 10000000
confirmed_sum_by_pop_inc = get_by_pop_inc_df(confirmed_sum,confirmed_factor)
deaths_sum_by_pop_inc = get_by_pop_inc_df(deaths_sum,deaths_factor)


#confirmed_sum_by_pop_inc.head(3)
#confirmed_100K = confirmed_sum.loc['Spain'].diff() / (pop_filtered.loc['Spain']['PopTotal'] /100)
#confirmed_100K.rolling(min_periods=1, window=7).sum().tail(35)



## Gráficos

Se obtienen gráficos para fallecimientos y casos confirmados utilizando ventanas deslizantes de
longitud variable (**confirmed_window** y **deaths_window**) para casos confirmados y fallecimientos.
Se ha comprobadoque los datos presentan fuerte correlación con el día de la semana. Por elllo,
Cualquier longitud de ventana que no sea múltiplo de 7 causa dientes de sierra.

Los países para los cuales se desean los gráficos están en el array **interesting_countries**.

El último dato es el de la fecha en la que se ejecuta el notebook (**todate = today**). El primero de 
los datos es el de hace **nweeks** (configurable) semanas.

La escala en y puede ser logarítimca (**logaritmic = True**) o no (**logaritmic = False**).

El tamaño del gráfico se configura con **fsize**. Se trata de una dupla en el que el primer
componente es el tamaño del eje x y el segundo el del eje y.


In [ ]:
import datetime as dt

nweeks=10
interesting_countries = [
                         'Spain',
                         'United Kingdom', 
                         #'Denmark',
                         #'Ireland', 
                         'Portugal',  
                         #'Israel',
                         'France',
                         'Germany',
                         'Italy',
                         #'Netherlands',
                         'Belgium',
                         #'Sweden',
                         #'US',
                         #'Chile',
                         #'Argentina',
                         #'Venezuela',
                         #'Japan',
                         #'Canada',
                         #'Australia',
                         #'Mexico',
                         #'Colombia',
                         #'Peru',
                         'Morocco',
                         #'India',
                         #'Russia',
                         #'South Africa',
                        ]

today = dt.date.today()
#yesterday = today - dt.timedelta(days=1)
fsize=(20,10)
logaritmic = False
confirmed_window = 14
deaths_window = 14
# fromdate=today - dt.timedelta(weeks=nweeks, days=max(confirmed_window, deaths_window))
fromdate=today - dt.timedelta(weeks=nweeks)

todate=today
#fsize=(10,5)
#fromdate = dt.datetime(2020,3,13)
#todate = dt.datetime(2020,4,3)
logaritmic= False


fromdate_df = fromdate - dt.timedelta(days=max(confirmed_window, deaths_window))

# deaths_sum_df = deaths_sum
# deaths_sum_df.columns = pd.to_datetime(deaths_sum_df.columns)
# deaths_sum_df.loc[['Spain','Italy','United Kingdom','Germany','Portugal']]


Los primeros gráficos son comparativos entre países. Permiten ver la evolución de la enfermedad
en distintos entornos

In [ ]:
cw_df = confirmed_sum_by_pop_inc[fromdate_df:todate].rolling(min_periods=1, window=confirmed_window).sum()[interesting_countries]
cw_df[fromdate:todate].plot(
                            title='Casos de últimos {} días por {} habitantes'.format(confirmed_window, confirmed_factor), 
                            figsize=fsize, 
                            logy=logaritmic,
                            grid=True
                           )

cw2_df = confirmed_sum_by_pop_inc[fromdate_df:todate].rolling(min_periods=1, window=int(confirmed_window/2)).sum()[interesting_countries]
cw2_df[fromdate:todate].plot(
                             title='Casos de últimos {} días por {} habitantes'.format(confirmed_window/2, confirmed_factor), 
                             figsize=fsize, 
                             logy=logaritmic,
                             grid=True
                            )
dw_df = deaths_sum_by_pop_inc[fromdate_df:todate].rolling(min_periods=1, window=deaths_window).sum()[interesting_countries]
dw_df[fromdate:todate].plot(
                            title='Fallecimientos en últimos {} días por {} habitantes'.format(deaths_window, deaths_factor), 
                            figsize=fsize, 
                            logy=logaritmic,
                            grid=True
                           )
dw2_df = deaths_sum_by_pop_inc[fromdate_df:todate].rolling(min_periods=1, window=int(deaths_window/2)).sum()[interesting_countries]
dw2_df[fromdate:todate].plot(
                             title='Fallecimientos en últimos {} días por {} de habitantes'.format(deaths_window/2, deaths_factor), 
                             figsize=fsize, 
                             logy=logaritmic,
                             grid=True
                            )

Curvas de vacunación

In [ ]:
vac_grouped = vaccinations.groupby('location')

suffixes = {
    'people_vaccinated_per_hundred': ' people',
    'people_fully_vaccinated_per_hundred': ' fully'
}

# Utilizamos los nombres de países de la Johns Hopkins.
# Creamos, para ello, un diccionario que traduce el nombre del país en los datos
# de vacunaciones al nombre en los datos de la Johns Hopkins

change_vac_countries = {
    'US': 'United States'
}

change_vac_dict = {}
for k in change_vac_countries.keys():
    for suffix in suffixes.values():
        change_vac_dict[k + suffix] = change_vac_countries[k] + suffix

# Creamos un nuevo dataframe iterando sobre todos los países en grouped,
# añadiendo, para cada país, tantas columnas como líneas hay en el diccionario
# suffixes utilizando, en lugar del nombre de la clave, el del país seguido por
# el sufijo correspondiente.

vc_df = pd.DataFrame()
        

interesting_vac_renamed_countries = []
for country in interesting_countries:
    if country in change_vac_countries.keys():
        vcname = change_vac_countries[country]
    else:
        vcname = country
 
    if vcname in vac_grouped.groups:
        interesting_vac_renamed_countries.append((country,vcname))

        vacgr_df = vac_grouped.get_group(vcname).set_index('date')

        for item in suffixes.keys():
            vc_df[country + suffixes[item]] = vacgr_df[item]
        

vc_df.fillna(method='ffill',inplace=True)

#vc_df.fillna(0.0, inplace=True)

#vc_df [['Spain people', 'Spain fully']]

vc_labels = []
for i in interesting_vac_renamed_countries:
    for suffix in suffixes.values():
        vc_labels.append(i[0] + suffix)

vc_df_toplot = vc_df[vc_labels]
    
vc_df_toplot[fromdate:todate].plot(
                      title='Vacunaciones (%)', 
                      figsize=fsize, 
                      logy=False,
                      grid=True
                     )

Los siguientes gráficos muestran los casos confirmados y fallecimientos por cada país.

Los casos confirmados en las ventanas pequeñas son predictores de los casos confirmados en
ventanas más grandes y ambos son predictores del número de fallecimientos

In [ ]:
for ic in interesting_countries:
    country_df = pd.DataFrame()
    country_df['Casos confirmados en {} en los últimos {} días'.format(ic,str(confirmed_window))] = cw_df[ic]
    country_df['Casos confirmados en {} en los últimos {} días'.format(ic,str(int(confirmed_window/2)))] = cw2_df[ic]
    country_df['Fallecimientos en {} en los últimos {} días'.format(ic,str(confirmed_window))] = dw_df[ic]
    country_df['Fallecimientos en {} en los últimos {} días'.format(ic,str(int(confirmed_window/2)))] = dw2_df[ic]
    if ic + suffixes['people_fully_vaccinated_per_hundred'] in vc_df:
        country_df['Vacunados completamente por mil habitantes']= vc_df[ ic + suffixes['people_fully_vaccinated_per_hundred']]*10
    if ic + suffixes['people_vaccinated_per_hundred'] in vc_df:
        country_df['Vacunados por mil habitantes']= vc_df[ic + suffixes['people_vaccinated_per_hundred']]*10

    country_df[fromdate:todate].plot(
                   title='Casos (por {} habitantes), fallecimientos (por {} habitantes) y vacunaciones (por 1000 habitantes) para {}'.format(confirmed_factor, deaths_factor, ic), 
                                               figsize=fsize, 
                                               logy=False,
                                               grid=True
                                              )



In [ ]:
# first_date = dt.date(2020,3,1)
# drange = pd.date_range(first_date,today)

# msr_pd = pd.read_csv(data_dir + '/' + measures_file)
# msr_pd = msr_pd[msr_pd['Country'] != 'Country']
# msr_pd = msr_pd.fillna(today.strftime('%Y-%m-%d'))

# msr_pd[['date_start', 'date_end']] = msr_pd[['date_start', 'date_end']].apply(pd.to_datetime)
# msr_pd['Response_measure'] = msr_pd['Response_measure'].astype('category')

# msr_countries = msr_pd['Country'].unique()
# responses = msr_pd['Response_measure'].unique()

# def is_between (d,first,last):
#     return (d>=first) and (d <= last) 
        

# msr_pc_pd = {}
# # msr_prov = msr_pd[msr_pd['Country'] == 'Spain']
# # msr_pc_pd['Spain'] = pd.DataFrame(index=drange, columns=responses).fillna(0)
# for country in interesting_countries:
#     if country in msr_countries:
#         msr_prov = msr_pd[msr_pd['Country'] == country]
#         msr_pc_pd[country] = pd.DataFrame(index=drange, columns=responses).fillna(0)
#         for _,row in msr_prov.iterrows():
#             for ix,_ in msr_pc_pd[country].iterrows():
#                 if is_between(ix,row['date_start'],row['date_end']):
#                     msr_pc_pd[country].loc[ix][row['Response_measure']] = 1

In [ ]:
# msr_plot_df = pd.DataFrame()
# for country in interesting_countries:
#     if country in msr_countries:
#         msr_plot_df[country] = msr_pc_pd[country].sum(axis='columns')

# msr_plot_df[fromdate:todate].plot(
#     title='Número de medidas tomadas',
#     figsize=fsize
# )